**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ASSAY=KS91_K562_ASTARRseq
ls ${FD_RES}/results/${ASSAY}/coverage/track_counts

KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep4.SUBSET.unstranded.bed.gz


In [3]:
ASSAY=KS91_K562_ASTARRseq

FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_counts
FP_INPS=($(ls ${FD_INP}/*rep*SUBSET*bed.gz))
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized

ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

echo "Total:  ${#FP_INPS[@]}"
for FP_INP in ${FP_INPS[@]}; do
    ###
    FN_INP=$(basename ${FP_INP})
    FN_OUT=${FN_INP}
    FP_OUT=${FD_OUT}/${FN_OUT}
    
    ###
    echo ${FN_INP}
done

Total:  10
KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep4.SUBSET.unstranded.bed.gz


In [4]:
FP_LIB=${FD_RES}/results/${ASSAY}/coverage/summary/library_size_summary.tsv
echo ${FP_LIB}
echo 
cat  ${FP_LIB}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage/summary/library_size_summary.tsv

FName	Size	Prefix	Sample	Group	Type	Genome
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz	348695063	KS91_K562_ASTARRseq	Input.rep1	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz	451369741	KS91_K562_ASTARRseq	Input.rep2	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz	487579055	KS91_K562_ASTARRseq	Input.rep3	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz	456246254	KS91_K562_ASTARRseq	Input.rep4	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz	444268950	KS91_K562_ASTARRseq	Input.rep5	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz	397333562	KS91_K562_ASTARRseq	Input.rep6	Input	raw	hg38
KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz	44103844	KS91_K562_ASTARRseq	Output.rep1	Output	raw	hg38
KS91_K562_ASTARRseq.Output.rep2.WGS.unstranded.bed.gz	97471282	KS91_K5

## RUN

In [5]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_perbase_normalize_subset_ASTARR.%a.txt \
    --array 0-9 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=KS91_K562_ASTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_counts
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized

FD_LIB=${FD_RES}/results/${ASSAY}/coverage/summary
FN_LIB=library_size_summary.tsv
FP_LIB=${FD_LIB}/${FN_LIB}

FP_INPS=($(ls ${FD_INP}/*rep*SUBSET*bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}
PREFIX=${FN_OUT/SUBSET/WGS}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_LIB}
echo
echo "show first few lines of file"
cat ${FP_LIB}
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_INP}
echo
echo "show first few lines of file"
zcat ${FP_INP} | head -5
echo

### execute
STRING1="\t\\."
STRING2="\t0"

zcat ${FP_INP} \
| sed --expression="s/${STRING1}/${STRING2}/g" \
| awk -v PREFIX=${PREFIX} '{print $1 "\t" $2 "\t" $3 "\t" $4 "\t" PREFIX}' \
| join -1 5 -2 1 -t $'\t' -o 1.1,1.2,1.3,1.4,2.2 - ${FP_LIB} \
| awk '{print $1 "\t" $2 "\t" $3 "\t" $4 * 10^6 / $5}' \
| gzip -c \
> ${FP_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574632


## Check

In [6]:
cat ${FD_LOG}/coverage_perbase_normalize_subset_ASTARR.0.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+13:51:55

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage/summary/library_size_summary.tsv

show first few lines of file
FName	Size	Prefix	Sample	Group	Type	Genome
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz	348695063	KS91_K562_ASTARRseq	Input.rep1	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz	451369741	KS91_K562_ASTARRseq	Input.rep2	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz	487579055	KS91_K562_ASTARRseq	Input.rep3	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz	456246254	KS91_K562_ASTARRseq	Input.rep4	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz	444268950	KS91_K562_ASTARRseq	Input.rep5	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz	397333562	KS91_K562_ASTARRseq	Input.rep6	Input	raw	hg38


In [7]:
cat ${FD_LOG}/coverage_perbase_normalize_subset_ASTARR.9.txt

Hostname:           x3-05-4.genome.duke.edu
Slurm Array Index:  9
Time Stamp:         03-15-23+13:51:55

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage/summary/library_size_summary.tsv

show first few lines of file
FName	Size	Prefix	Sample	Group	Type	Genome
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz	348695063	KS91_K562_ASTARRseq	Input.rep1	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz	451369741	KS91_K562_ASTARRseq	Input.rep2	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz	487579055	KS91_K562_ASTARRseq	Input.rep3	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz	456246254	KS91_K562_ASTARRseq	Input.rep4	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz	444268950	KS91_K562_ASTARRseq	Input.rep5	Input	raw	hg38
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz	397333562	KS91_K562_ASTARRseq	Input.rep6	Input	raw	hg38
